<a href="https://colab.research.google.com/github/JackTheProgrammer/Research-purpose-AI-agent/blob/main/ai_agent_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Research purpose AI agent from scratch using Google Gemini

## Installing libraries

In [ ]:
!pip install -r /content/.config/requirements.txt

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.

## Tools

In [ ]:
from langchain.tools import Tool
from langchain_community.tools import (
    WikipediaQueryRun,
    DuckDuckGoSearchRun,
    ArxivQueryRun
)
from langchain_community.utilities import (
   WikipediaAPIWrapper,
   ArxivAPIWrapper
)
from datetime import datetime

arxiv_wrapper = ArxivAPIWrapper(load_max_docs=3)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

wikpedia_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=100
)
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=wikpedia_wrapper
)

ddg = DuckDuckGoSearchRun()
ddg_tool = Tool(
    name="ddg_search",
    description="Search DuckDuckGo for the relevant results",
    func=ddg.run
)

def save_to_txt(data: str, filename: str = "research_output.txt"):
  timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  formatted_text = f"--- Research Output ---\nTimestamp: {timestamp}\n\n{data}\n\n"

  with open(filename, "a", encoding="utf-8") as f:
    f.write(formatted_text)

  return f"Data successfully saved to {filename}"

file_saving_tool = Tool(
    name="save_to_text_file",
    description="Save the research output to a text file",
    func=save_to_txt
)

tools = [
    arxiv_tool,
    wikipedia_tool,
    ddg_tool,
    file_saving_tool
]

## Agent

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('/content/.config/env'))

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [ ]:
class ResponseTemplate(BaseModel):
  topic: str
  summary: str
  sources: list[str]
  tools_used: list[str]

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    google_api_key=os.environ['GEMINI_API_KEY']
)
parser = PydanticOutputParser(pydantic_object=ResponseTemplate)

message = [
    (
        "system",
        """
        You are a research assistant that will help generate a research paper.
        Answer the user query and use neccessary tools.
        Wrap the output in this format and provide no other text\n{format_instructions}
        """
    ),
    ("placeholder", "{chat_history}"),
    ("human", "{query}"),
    ("placeholder", "{agent_scratchpad}")
]

prompt = ChatPromptTemplate.from_messages(
    messages=message,
).partial(
    format_instructions=parser.get_format_instructions()
)

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
query = input("Enter your research topic ")
raw_response = agent_executor.invoke(
    {
        "query": query
    }
)

raw_response

Enter your research topic Ethical discrepancies and ironies of NPD patients


> Entering new AgentExecutor chain...
```json
{"topic": "Ethical Discrepancies and Ironies of Narcissistic Personality Disorder (NPD)", "summary": "Narcissistic Personality Disorder (NPD) presents several ethical discrepancies and ironies. Individuals with NPD often lack empathy and exploit others, leading to ethical concerns about their interpersonal relationships and societal impact. Ironically, despite their grandiosity and sense of entitlement, they are often deeply insecure and vulnerable. This internal conflict creates a discrepancy between their perceived self and their actual self, leading to further ethical complexities. The disorder also raises questions about moral responsibility, as the impaired capacity for empathy and moral reasoning may mitigate culpability for unethical behavior. Further research is needed to fully understand the ethical dimensions of NPD.", "sources": ["Wikipedia: Narcissisti

{'query': 'Ethical discrepancies and ironies of NPD patients',
 'output': '```json\n{"topic": "Ethical Discrepancies and Ironies of Narcissistic Personality Disorder (NPD)", "summary": "Narcissistic Personality Disorder (NPD) presents several ethical discrepancies and ironies. Individuals with NPD often lack empathy and exploit others, leading to ethical concerns about their interpersonal relationships and societal impact. Ironically, despite their grandiosity and sense of entitlement, they are often deeply insecure and vulnerable. This internal conflict creates a discrepancy between their perceived self and their actual self, leading to further ethical complexities. The disorder also raises questions about moral responsibility, as the impaired capacity for empathy and moral reasoning may mitigate culpability for unethical behavior. Further research is needed to fully understand the ethical dimensions of NPD.", "sources": ["Wikipedia: Narcissistic personality disorder", "Arxiv: Narciss

|Platform | Links |
|---------|--------|
|GitHub|[JackTheProgrammer](https://github.com/JackTheProgrammer/)|
|LinkedIn |[Fawad Awan](https://pk.linkedin.com/in/fawad-awan-893a58171)|